# Jackknife Resampling Tests

- Divide observations into 'patches'
- Leave out 1 patch and calculate parameter using the rest of the patches 

In [2]:
import des_functions
import treecorr
import numpy as np
import astropy.io.fits as fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.stats.jackknife as jackknife
import matplotlib.pyplot as plt
from more_itertools import locate
data_dir = '/home/b7009348/projects/WGL_Project/DES-data/'
treecorr.__version__

'4.2.0'

In [3]:
# paths to required files
data_dir = '/home/b7009348/projects/WGL_Project/DES-data/'
lens_file = 'DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'
source_file = 'y1_im3shape_z=0.9-1.3.fits'
source_Zs = 'y1a1-gold-mof-badregion_BPZ.fits'

### Jackknife using indexing on patches attribute of catalogues

In [ ]:
# define bin and scale params
nbins=20
npatches=20
theta_min=2.5
theta_max=250

# list of source files to compute cross-correlation for 
source_files = source_file

# read in lens data and create treecorr catalogue object to store
with fits.open(data_dir+lens_file) as hdu:
    data = hdu[1].data
    ra_l = data['RA']
    dec_l = data['DEC']
    w_l = data['weight']
    
# pass in npatch arguement to create catalogue using patches    
cat_l = treecorr.Catalog(ra=ra_l, dec=dec_l, ra_units='deg', dec_units='deg', w=w_l, npatch=npatches)

del data, ra_l, dec_l, w_l

# read in randoms data and create treecorr catalogue
with fits.open(data_dir+'DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits') as hdu:
    data = hdu[1].data
    #indexes = list(locate(data['z'], lambda x: 0.3 < x < 0.45)) # cut randoms to same redshift as lenses 
    ra_r = data['RA']#[indexes]
    dec_r = data['dec']#[indexes]
    
# define patches based on patches from lens catalogue so all catalogues use the same patches    
cat_r = treecorr.Catalog(ra=ra_r, dec=dec_r, ra_units='deg', dec_units='deg', patch_centers=cat_l.patch_centers)

del data, ra_r, dec_r

#del indexes

with fits.open(data_dir+source_files) as hdu:
    data = hdu[1].data
    ra_s = data['ra']
    dec_s = data['dec']
    e1 = data['e1'] - data['c1']
    e2 = data['e2'] - data['c2']
    sens = data['m'] + 1.0
    w = data['weight']

    
cat_s = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', g1=e1, g2=e2, w=w, patch_centers=cat_l.patch_centers)
cat_k = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', k=sens, w=w, patch_centers=cat_l.patch_centers)

del data, ra_s, dec_s, w, e1, e2, sens

print('Total catalogues created, proceeding with jackknife...')

gammaT_patches = np.zeros([npatches, nbins])
theta_patches = np.zeros_like(gammaT_patches)
boost_patches = np.zeros_like(gammaT_patches)

#loop to exclude 1 patch at a time
for i in range(npatches):
    
    #locate patch in catlogues
    l_indexes = list(locate(cat_l.patch, lambda x: x != i))
    r_indexes = list(locate(cat_r.patch, lambda x: x != i))
    s_indexes = list(locate(cat_s.patch, lambda x: x != i))
    k_indexes = list(locate(cat_k.patch, lambda x: x != i))   
    
    print('Patch %g located...' %i)
    
    #create new catalogues with this patch missing
    jk_l = treecorr.Catalog(ra=cat_l.ra[l_indexes], dec=cat_l.dec[l_indexes], ra_units='rad', dec_units='rad', w=cat_l.w[l_indexes])
    
    jk_r = treecorr.Catalog(ra=cat_r.ra[r_indexes], dec=cat_r.dec[r_indexes], ra_units='rad', dec_units='rad', w=cat_r.w[r_indexes])
    
    jk_s = treecorr.Catalog(ra=cat_s.ra[s_indexes], dec=cat_s.dec[s_indexes], ra_units='rad', dec_units='rad', g1=cat_s.g1[s_indexes], 
                                g2=cat_s.g2[s_indexes], w=cat_s.w[s_indexes])
    
    jk_k = treecorr.Catalog(ra=cat_k.ra[k_indexes], dec=cat_k.dec[k_indexes], ra_units='rad', dec_units='rad', k=cat_k.k[k_indexes], 
                                w=cat_k.w[k_indexes])
    
    del l_indexes, r_indexes, s_indexes, k_indexes
    
    print('Number of sources in patch: %f' %jk_s.nobj)
    print('Total number of sources: %f' %cat_s.nobj)
    
    # do correlations with lenese, NKCorrelation is used to apply multiplicative correction
    ng = treecorr.NGCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    ng.process(jk_l, jk_s)
    nk = treecorr.NKCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    nk.process(jk_l, jk_k)
    
    # do correlations with randoms
    rg = treecorr.NGCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    rg.process(jk_r, jk_s)
    rk = treecorr.NKCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    rk.process(jk_r, jk_k)
    
    # do count correlations to find boost
    ls = treecorr.NNCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    ls.process(jk_l, jk_s)

    rs = treecorr.NNCorrelation(nbins=nbins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin')
    rs.process(jk_r, jk_s)
    
    print('Correlation calculation complete, saving data...')
    
    # correlation functions and corrections for lenses and randoms
    xi_l = ng.xi
    sens_l = nk.xi
    xi_r = rg.xi
    sens_r = rk.xi
    
    # store data in preallocated arrays
    gammaT_patches[i,:] = xi_l/sens_l - xi_r/sens_r
    theta_patches[i,:] = np.exp(ng.meanlogr)
    
    nrand = jk_r.nobj
    nlens =jk_l.nobj
    
    boost_patches[i,:] = nrand/nlens * ls.weight/rs.weight
    
    
gammaT = np.zeros([nbins])
var = np.zeros_like(gammaT)
boost = np.zeros_like(gammaT)
boost_var = np.zeros_like(gammaT)
for i in range(nbins):
    
    # rows represent patches, columns ang. sep. bins
    gammaT[i] = 1.0/npatches * np.sum(gammaT_patches[:,i]) # sum over all patches
    boost[i] = 1.0/npatches * np.sum(boost_patches[:,i])
    
    # each row of GammaT_patches corresponds to an x_i estimate
    var[i] = 1.0/(npatches-1) * np.sum((gammaT_patches[:,i] - gammaT[i])**2) # sum over x_i - x
    boost_var[i] = 1.0/(npatches-1) * np.sum((boost_patches[:,i] - boost[i])**2)
    
    # calculate average theta for all patches
    theta[i] = 1.0/npatches * np.sum(theta_patches[:,i])
    
sig = np.sqrt(var)
boost_sig = np.sqrt(boost_var)

Total catalogues created, proceeding with jackknife...
Patch 0 located...
Number of sources in patch: 1941073.000000
Total number of sources: 2013303.000000
Correlation calculation complete, saving data...
Patch 1 located...
Number of sources in patch: 1936885.000000
Total number of sources: 2013303.000000
Correlation calculation complete, saving data...
Patch 2 located...


In [1]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['cmr10']
plt.rcParams['mathtext.fontset'] ='cm'

plt.figure(figsize=[6,5])
plt.errorbar(theta, gammaT, yerr=sig, lw=0, marker='o', color='orange', markersize=5, elinewidth=1, capsize=3)
plt.yscale('log')
plt.xscale('log')
plt.ylim([1e-6, 1e-2])
plt.xlim([2.5,250])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\gamma_{t}(\theta)$')
plt.title(r'$0.30<z_{l}<0.45$')
plt.savefig('jackknife_zl=0.3-0.45_zs=all.png', dpi=300)

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=[6,5])
plt.errorbar(theta_final, boost, yerr=boost_sig, lw=0, marker='o', color='orange', markersize=5, elinewidth=1, capsize=3)
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'B($\theta$)')
plt.title(r'$0.30 < z_{l} < 0.45$')
plt.legend([r'$0.20<z_{s}<0.43$', r'$0.43<z_{s}<0.63$', r'$0.63<z_{s}<0.90$', r'$0.90<z_{s}<1.30$'])
plt.xscale('log')
plt.ylim([0.98,1.12])
plt.xlim([2.5,250])
plt.savefig('jackknife_boosts_zl=0.3-0.45.png')

### Jackknife using manual definition of patches

- patch centers stored as (x,y,z)

In [72]:
print(cat_l.patch_centers)

counts = 0
for value in cat_l.patch:
    
    if value == 0:
        counts = counts +1 
        
print(counts)

[[ 0.35413527 -0.44633383 -0.82181161]
 [ 0.47774971 -0.31716729 -0.81924363]
 [ 0.54539164 -0.1756263  -0.81957511]
 [ 0.62222779 -0.05689724 -0.78076583]
 [ 0.68611473 -0.17974666 -0.70493809]
 [ 0.58757677  0.0921854  -0.80390011]
 [ 0.71597824  0.03541437 -0.69722377]
 [ 0.48178334  0.32674835 -0.81309306]
 [ 0.53787712  0.20602692 -0.81746016]
 [ 0.67541167  0.20602494 -0.70807683]
 [ 0.00345964  0.59021241 -0.80724057]
 [ 0.1566531   0.58139513 -0.79839809]
 [ 0.01338874  0.71279091 -0.70124878]
 [ 0.29452347  0.5066864  -0.81026219]
 [ 0.16202126  0.70057231 -0.69494428]
 [ 0.39362654  0.41441345 -0.82056057]
 [ 0.61756149  0.34990967 -0.70440119]
 [ 0.29302637  0.64039529 -0.7099503 ]
 [ 0.42368534  0.5705177  -0.70356257]
 [ 0.52842607  0.46623903 -0.70949775]]
33602


In [83]:
cart_patch = cat_l.patch_centers

print(cart_patch)

icrs_patch = SkyCoord(x=cart_patch[:,0], y=cart_patch[:,1], z=cart_patch[:,2], unit='mpc', representation_type='cartesian')

icrs_patch.representation_type = 'unitspherical'

print(icrs_patch)

[[ 0.35413527 -0.44633383 -0.82181161]
 [ 0.47774971 -0.31716729 -0.81924363]
 [ 0.54539164 -0.1756263  -0.81957511]
 [ 0.62222779 -0.05689724 -0.78076583]
 [ 0.68611473 -0.17974666 -0.70493809]
 [ 0.58757677  0.0921854  -0.80390011]
 [ 0.71597824  0.03541437 -0.69722377]
 [ 0.48178334  0.32674835 -0.81309306]
 [ 0.53787712  0.20602692 -0.81746016]
 [ 0.67541167  0.20602494 -0.70807683]
 [ 0.00345964  0.59021241 -0.80724057]
 [ 0.1566531   0.58139513 -0.79839809]
 [ 0.01338874  0.71279091 -0.70124878]
 [ 0.29452347  0.5066864  -0.81026219]
 [ 0.16202126  0.70057231 -0.69494428]
 [ 0.39362654  0.41441345 -0.82056057]
 [ 0.61756149  0.34990967 -0.70440119]
 [ 0.29302637  0.64039529 -0.7099503 ]
 [ 0.42368534  0.5705177  -0.70356257]
 [ 0.52842607  0.46623903 -0.70949775]]
<SkyCoord (ICRS): (ra, dec) in deg
    [(308.42958787, -55.26655587), (326.4207104 , -55.00914965),
     (342.15047124, -55.04228323), (354.77533654, -51.33074797),
     (345.31970424, -44.824543  ), (  8.91649634, -53.

In [107]:
ra_patch = icrs_patch.frame.ra.deg
dec_patch = icrs_patch.frame.dec.deg
print(ra_patch)

[308.42958787 326.4207104  342.15047124 354.77533654 345.31970424
   8.91649634   2.83170808  34.1453557   20.9586957   16.9635984
  89.66415439  74.92015146  88.92390837  59.83169049  76.97815241
  46.473615    29.53584416  65.41253677  53.40119733  41.42249087]
-17.99112252232561


In [ ]:
# load in redmagic photo-z values for all lenses
with fits.open(data_dir+'DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits') as hdu:
    data = hdu[1].data
    lens_z = data['z']
del data

cat_l = treecorr.Catalog(r=lens_z npatch=npatches)

In [ ]:
# load in source IDs for matched im3shape catalogue
with fits.open(data_dir+'y1_im3_shapes_matched.fits') as hdu:
    data = hdu[1].data
    im3_ID = data['coadd_objects_id']
    im3_weight = data['weight']
del data

# load in z_mc for all sources
with fits.open(data_dir+'y1a1-gold-mof-badregion_BPZ.fits') as hdu:
    data = hdu[1].data
    im3_zmc = data['Z_MC']
    zmc_ID = data['COADD_OBJECTS_ID']
del data

# match z_mc values to source IDs
matches, zmc_indices, shape_indices = np.intersect1d(zmc_ID, im3_ID, return_indices=True)

print(np.size(zmc_indices), np.size(im3_ID))
print(zmc_indices)

# slice z_mc values to only to those in matched catalogue
im3_zmc = im3_zmc[zmc_indices]

# delete uneccessary arrays
del matches, zmc_indices, shape_indices, zmc_ID

